# Ocean rendering

## Previous work

### Oceans
Tessendorf FFT in real time, Wave particles 

### Simulation
Positional Based Fluids,Macklin, Müller, SIGGRAPH 2013


## References
* https://www.fxguide.com/featured/assassins-creed-iii-the-tech-behind-or-beneath-the-action/
* http://advances.realtimerendering.com/s2017/Ocean_SIGGRAPH17_Final.pptx
* https://github.com/huwb/crest-oceanrender
* http://advances.realtimerendering.com/s2016/Rendering%20rapids%20in%20Uncharted%204.pptx
* http://advances.realtimerendering.com/s2013/OceanShoestring_SIGGRAPH2013_Online.pptx  
* https://ttnghia.github.io/portfolio/project-ocean-surface-simulation/

## Examples

### Gerstner

In [ ]:
import numpy as np
import ipyvolume as ipv

def gerstner(t, x0, y0, direction, wave_length, amplitude, phase):
    k = 2 * np.pi / wave_length
    wave_dir = np.array([np.cos(direction), np.sin(direction)])
    wave_vector = wave_dir * k
    frequency = np.sqrt(9.8*k)
    calc = x0*wave_vector[0] + y0*wave_vector[1] - frequency * t + phase
    ox = -wave_dir[0] * amplitude * np.sin(calc)
    oy = -wave_dir[1] * amplitude * np.sin(calc)
    z = amplitude * np.cos(calc)
    return ox, oy, z

u = np.linspace(0, 8, 256)
v = np.linspace(0, 8, 256)
x, y = np.meshgrid(u, v)
z = np.zeros_like(x)
ox, oy, oz = gerstner(1, x, y, np.pi*0.25, 3, 0.2, np.pi/2)
x += ox
y += oy
z += oz
ox, oy, oz = gerstner(1, x, y, np.pi*0.35, 4, 0.4, np.pi)
x += ox
y += oy
z += oz


# Plot the surface
ipv.figure()
ipv.xlim(0, 8)
ipv.zlim(0, 8)
ipv.ylim(-4, 4)
ipv.plot_surface(x, z, y, color="Grey")
ipv.show()

### Tessendorf FFT

In [ ]:
import ipyvolume as ipv
import numpy as np

In [ ]:
def func_p_h(wavevector_x, wavevector_z):
    # add a small bias to avoid divide by 0
    k = np.sqrt(wavevector_x**2 + wavevector_z**2)+0.000001
    L = wind_speed**2/gravitational_constant
    cosine_factor = (wavevector_x/k*wind_dir[0] + wavevector_z/k*wind_dir[1])**2
    return A * np.exp(-1/(k*L)**2) / k**4 * cosine_factor

In [ ]:
def func_h_twiddle_0(wavevector_x, wavevector_z):
    size = np.size(wavevector_x)
    xi_r = np.random.normal(0, 1, size)
    xi_i = np.random.normal(0, 1, size)
    xi_r = np.reshape(xi_r, wavevector_x.shape)
    xi_i = np.reshape(xi_i, wavevector_x.shape)
    return 1/np.sqrt(2) * (xi_r + 1j*xi_i) * np.sqrt(func_p_h(wavevector_x, wavevector_z))

def func_h_twiddle(h_twiddle_0, h_twiddle_0_conj, wavevector_x, wavevector_z, t):
    k = np.sqrt(wavevector_x**2 + wavevector_z**2)
    w = np.sqrt(k*gravitational_constant)
    return h_twiddle_0*np.exp(1j*w*t) + h_twiddle_0_conj*np.exp(-1j*w*t)

In [ ]:
N = 128
L = 1000 # m
A = 3e-7
gravitational_constant = 9.80665 # m/s^2
wind_speed = 30 # m/s
wind_dir = (1,1)
wind_dir /= np.linalg.norm(wind_dir)
lambda_ = 1

k = np.linspace(0, N-1, N)
kx = (2*np.pi*k - np.pi*N)/L
kz = (2*np.pi*k - np.pi*N)/L
wavevector_x, wavevector_z = np.meshgrid(kx, kz)
grid_N, grid_M = np.meshgrid(k, k)

h_twiddle_0 = func_h_twiddle_0(wavevector_x, wavevector_z)
h_twiddle_0_conj = np.conjugate(func_h_twiddle_0(-wavevector_x, -wavevector_z))


x = k * L / N
z = k * L / N
x, z = np.meshgrid(x, z)
xs = []
zs = []
ys = []
for t in np.linspace(0, 40, 48):
    sign = (-1)**(grid_N + grid_M)
    scale = N * N
    h_twiddle = func_h_twiddle(h_twiddle_0, h_twiddle_0_conj, wavevector_x, wavevector_z, t)
    ys.append(np.fft.ifft2(h_twiddle).real * sign * scale)
    
    k = np.sqrt(wavevector_x**2 + wavevector_z**2) + 0.0000001
    xs.append(x + lambda_ * np.fft.ifft2(-1j * wavevector_x / k * h_twiddle).real * sign * scale)
    zs.append(z + lambda_ * np.fft.ifft2(-1j * wavevector_z / k * h_twiddle).real * sign * scale)

ipv.figure()
ipv.xlim(0, L)
ipv.zlim(0, L)
ipv.ylim(-L/2, L/2)
s0 = ipv.plot_surface(xs, ys, zs, color="Grey")
s1 = ipv.plot_wireframe(xs, ys, zs)
ipv.animation_control((s0, s1)) # shows controls for animation controls
# ipv.animation_control(s1) # shows controls for animation controls
ipv.show()

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(xs[0]-x, cmap="gray")
plt.colorbar();
plt.show()